In [1]:
!pip install dash
!pip install dash_bootstrap_components
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.2 MB/s eta 0:00:00


In [7]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
import requests
import json
from loguru import logger
import os

# Initialize the app - incorporate a Dash Bootstrap theme
external_stylesheets = [dbc.themes.CERULEAN]

# app server
app = Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

# PREDICTION API URL
api_url = os.getenv('API_URL')
api_url = "http://{}:8001/api/v1/predict".format(api_url)


# App layout
app.layout = dbc.Container([
    dbc.Row([
      html.Div(children="ELECTRO DUNAS", className="d-block p-2 bg-primary text-white fs-1")
    ]),
    dbc.Row([
      html.Div(children="Consumo de electricidad segmento industrial", className="d-block p-2 bg-dark text-white fs-3")
    ]),
        dbc.Row([
      html.Div(children="Detección de perdidas no técnicas", className="d-block p-2 bg-dark text-white fs-5")
    ]),
    dbc.Row([
        dbc.Col([
          dbc.Label("Cliente:", className="fw-normal"),
          dcc.Dropdown(id='cliente-dropdown', options=['CLIENTE1', 'CLIENTE2', 'CLIENTE3', 'CLIENTE4', 'CLIENTE5', 'CLIENTE6', 'CLIENTE7', 'CLIENTE8', 'CLIENTE9', 'CLIENTE10'], value='CLIENTE1')
        ], width=2),
        dbc.Col([
          dbc.Label("Acumular por período:", className="fw-normal"),
          dcc.Dropdown(id='medida-dropdown', options=['Mínimo', 'Máximo', 'Media', 'No acumular'], value='Media')
        ], width=2),
        dbc.Col([
          dbc.Label("Período:", className="fw-normal"),
          dcc.Dropdown(
              id="periodo-input",
              options=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], value='1')
        ], width=1),
        dbc.Col([
          dbc.Label("Energia activa:", className="fw-normal"),
          dcc.Input(
              id="energia-input",
              placeholder="Ingrese la energía activa",
              max=0,
              min=100,
              step=0.01
          )
        ], width=2),
        dbc.Col([
            html.Div(id="resultado-output", className="fs-5 fw-bold")
        ], width=2)
    ]),
    dbc.Row([
        dbc.Col([
            html.Div(id="consumo-table")
        ], width=4),
        dbc.Col([
            dcc.Graph(figure={}, id='consumo-graph')
        ], width=6),
    ]),
  ], fluid=True)

@app.callback(
    [Output("consumo-table", "children"),
    Output(component_id='consumo-graph', component_property='figure')],
    [Input("cliente-dropdown", "value"),
     Input("medida-dropdown", "value")]
)
def update_output(cliente, medida):
    url = f'https://raw.githubusercontent.com/juanrave/electrodunas/main/datos/DATOS{cliente}.csv'
    df = pd.read_csv(url, sep=',')

    df['Periodo'] = df['Fecha'].str.slice(0, 7).str.replace("-", "")

    match medida:
      case 'Mínimo':
        data = df.groupby(['Periodo'])['Active_energy'].min()
        data = data.to_frame()
        data = data.reset_index()
        data.columns = ['Periodo', 'Active_energy']
      case 'Máximo':
        data = df.groupby(['Periodo'])['Active_energy'].max()
        data = data.to_frame()
        data = data.reset_index()
        data.columns = ['Periodo', 'Active_energy']
      case "Media":
        data = df.groupby(['Periodo'])['Active_energy'].mean()
        data = data.to_frame()
        data = data.reset_index()
        data.columns = ['Periodo', 'Active_energy']
      case _:
        data = df[['Periodo', 'Active_energy']]

    fig = px.line(data, x='Periodo', y='Active_energy')

    return html.Div(
        [
            dash_table.DataTable(
                data=data.to_dict("records"),
                columns=[{"id": x, "name": x} for x in data.columns],
                page_size= 10,
            )
        ]
    ), fig

# Method to update prediction
@app.callback(
    Output("resultado-output", "children"),
    [Input("energia-input", "value"),Input("periodo-input", "value")]
)
def update_output(energia, periodo):
    api_url = 'http://3.88.33.0:8001/getPredictionOutput'
    myreq = [
              {
                "Month": periodo,
                "Active_energy": energia
              }
            ]
    headers =  {"Content-Type":"application/json", "accept": "application/json"}

    # POST call to the API
    response = requests.post(api_url, data=json.dumps(myreq), headers=headers)
    data = response.json()
    logger.info("Response: {}".format(data))

    # Pick result to return from json format
    result = data['predict']

    return result

if __name__ == "__main__":
    logger.info("Running dash")
    app.run_server(debug=False)

2023-11-28 01:03:37.345 | INFO     | __main__:<cell line: 140>:141 - Running dash


<IPython.core.display.Javascript object>